In [2]:
# The first cell contains only code to generate the maze 
# The GA algorithm starts from the second cell
import random
from unittest import result
import numpy as np
import math

class Maze:
    
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.solutions = []
        self.empty = set()
        self.obstacle_loc = set()
        self.goal_state = None
        self.start_state = None
        self.start_state = (0, 0)
        self.solution_in_steps = 0
        self.generate_maze()
    
    def generate_maze(self):

        a_height = 2*self.height+1
        a_width = 2*self.width+1
        
        self.maze = []
        for i in range(a_height):
            if i%2 == 0:
                self.maze.append([1]*a_width)
            else:
                row = []
                for j in range(a_width):
                    if j%2 == 0:
                        row.append(1)
                    else:
                        row.append(0)
                self.maze.append(row)
        self.maze[1][1] = 2
        self.start_state = (1, 1)
        self.create_paths()
        self.set_goal_state()
    
    def create_paths(self):
        path = [(0, 0)]
        pos = (1, 1)
        steps_count = 0
        visited = {(1, 1)}
        while(len(visited)<self.height*self.width):
            possible_moves = self.generate_moves(pos,visited)
            if possible_moves == []:
                pos = path.pop()
                if pos == (1, 1):
                    break
                steps_count -= 1
                continue
            pos,wall = random.choice(possible_moves) 
            steps_count += 1
            path.append(pos)
            visited.add(pos)
            self.maze[wall[0]][wall[1]] = 0
        self.solution_in_steps = steps_count
    
    def set_goal_state(self):
        a_height = 2*self.height+1
        a_width = 2*self.width+1
        self.goal_state = (a_height-2, a_width-2)
        self.maze[a_height-2][a_width-2] = 3
             
    def generate_moves(self, pos, visited):
        possible_moves = []
        a_height = 2*self.height+1
        a_width = 2*self.width+1
        # moving up
        if pos[0] > 1 and (pos[0]-2, pos[1]) not in visited :
            possible_moves.append([(pos[0]-2, pos[1]), (pos[0]-1, pos[1])])
        # moving down
        if pos[0] < a_height-2 and (pos[0]+2, pos[1]) not in visited:
            possible_moves.append([(pos[0]+2, pos[1]), (pos[0]+1, pos[1])])
        # moving left
        if pos[1] > 1 and (pos[0], pos[1]-2) not in visited:
            possible_moves.append([(pos[0], pos[1]-2), (pos[0], pos[1]-1)])
        # moving right
        if pos[1] < a_width-2 and (pos[0], pos[1]+2) not in visited :
            possible_moves.append([(pos[0], pos[1]+2), (pos[0], pos[1]+1)])
        return possible_moves
  
    def show_path(self, path):
        
        a_width = 2*self.width+1
        pos = self.start_state
        self.empty = set()
        for next in path:
            if next == 0:
                pos = (pos[0], pos[1])
            elif next == 1:
                if self.maze[pos[0]-1][pos[1]] == 1:
                    continue
                pos = (pos[0]-1, pos[1])
            elif next == 2:
                if self.maze[pos[0]][pos[1]-1] == 1:
                    continue
                pos = (pos[0], pos[1]-1)
            elif next == 3:
                if self.maze[pos[0]+1][pos[1]] == 1:
                    continue
                pos = (pos[0]+1, pos[1])
            elif next == 4:
                if self.maze[pos[0]][pos[1]+1] == 1:
                    continue
                pos = (pos[0], pos[1]+1)
            if pos == self.goal_state:
                break
            self.empty.add(pos[0]*a_width+pos[1])
        print(self.__str__(show_path = True))
    def __str__(self,show_path = False):
        result = ''
        a_width = 2*self.width+1
        for i in range(len(self.maze)):
            for j in range(len(self.maze[0])):
                if self.maze[i][j] == 3:
                    result += 'E '
                elif self.maze[i][j] == 2:
                    result += 'S '
                elif self.maze[i][j] == 1:
                    result += '# '

                elif show_path and i*a_width+j in self.empty:
                    result += '0 '
                elif self.maze[i][j] == 0:
                    result += '  '

                
            result += '\n'
        return result


In [3]:
# After lots of testing 💻 , my GA would work as expected for maze of size 4x4 (i.e 9x9 board ) in reasonable amount of time (< 5min )
#  bigger the size of board more generations, popluation & time is required
maze_height = 4
maze_width = 4
maze = Maze(maze_height, maze_width)
maze.generate_maze()
print(maze)

# # # # # # # # # 
# S #           # 
#   # # # # #   # 
#       #       # 
# # #   #   # # # 
#   #   #       # 
#   #   # # #   # 
#             E # 
# # # # # # # # # 



In [4]:
# generates initial set of populations 🌆
def generate_init_population(n_population, steps):
    population = []
    for i in range(n_population):
        route = []
        for _ in range(steps):
            route.append(random.randint(0,4))
        population.append(route)
    return population

In [5]:
# gets fitness values of every element in the population & returns as list 🏋️‍♂️
def get_fitness(population, maze):
    fitness = []
    for route in population:
        fitness.append(get_route_fitness(route, maze))
    return fitness

In [6]:
# calcluates fitness of each element in population the fitness is calculated as follow
# score = 0
#   everytime we hit a wall the score is increased by penalty ⚽️
#   once we traverse through the route we calculate the distance of our current position with goal position 
#   To find right set of parents we traverse through route from end position and after traversal we calculate
#   the distance between current position and start point
#   !! The lesser the fitness value the better the route !!
#   Fitness = 0 signifies path is found with no collision to wall
def get_route_fitness(route,maze):
    score = 0
    visited = set()
    penalty = 0.5
    pos = maze.start_state
    for next in route:
        if next == 0:
            pos = (pos[0], pos[1])
            continue
        elif next == 1:
            if maze.maze[pos[0]-1][pos[1]] == 1:  # its a wall
                score += penalty                  # adding penality
                continue
            pos = (pos[0]-1, pos[1])
        elif next == 2:
            if maze.maze[pos[0]][pos[1]-1] == 1:
                score += penalty
                continue
            pos = (pos[0], pos[1]-1)
        elif next == 3:
            if maze.maze[pos[0]+1][pos[1]] == 1:
                score += penalty
                continue
            pos = (pos[0]+1, pos[1])
        elif next == 4:
            if maze.maze[pos[0]][pos[1]+1] == 1:
                score += penalty
                continue
            pos = (pos[0], pos[1]+1)
        if pos in visited:
            score += penalty
        if pos == maze.goal_state:
            return score
            
    # finding distance between the goal state and current positions
    score+=abs(pos[0]-maze.goal_state[0])+abs(pos[1]-maze.goal_state[1])
    pos = maze.goal_state
    for next in reversed(route):            # traversing from the end 
        if next == 0:
            pos = (pos[0], pos[1])
            continue
        elif next == 1:
            if maze.maze[pos[0]-1][pos[1]] == 1:
                score += penalty
                continue
            pos = (pos[0]-1, pos[1])
        elif next == 2:
            if maze.maze[pos[0]][pos[1]-1] == 1:
                score += penalty
                continue
            pos = (pos[0], pos[1]-1)
        elif next == 3:
            if maze.maze[pos[0]+1][pos[1]] == 1:
                score += penalty
                continue
            pos = (pos[0]+1, pos[1])
        elif next == 4:
            if maze.maze[pos[0]][pos[1]+1] == 1:
                score += penalty
                continue
            pos = (pos[0], pos[1]+1)
        if pos in visited:
            score += penalty
        if pos == maze.start_state:
            return score
    # finding distance between the start state and current positions
    score+=abs(pos[0]-maze.start_state[0])+abs(pos[1]-maze.start_state[1])
    return score

In [7]:
# getting next generation 👨‍👩‍👧‍👦👩‍👩‍👦‍👦
#  Here I tried different no of parents for each offspring starting from 2 to 4
#  I found pattern more the parents better the performance
def get_next_generation(population):
    next_generation = []
    for i in range(n_population - len(population)):
        parent_1 = random.choice(population)
        parent_2 = random.choice(population)
        parent_3 = random.choice(population)
        parent_4 = random.choice(population)
        cross_over_point_1 = random.randint(0, len(parent_1))
        cross_over_point_2 = random.randint(cross_over_point_1, len(parent_2))
        cross_over_point_3 = random.randint(cross_over_point_2, len(parent_3))
        offspring = parent_1[:cross_over_point_1] + parent_2[cross_over_point_1:cross_over_point_2] + parent_3[cross_over_point_2:cross_over_point_3] + parent_4[cross_over_point_3:]
        offspring = mutate(offspring)
        next_generation.append(offspring)
    return next_generation

In [8]:
# Mutating the offspring 👶
# Here I thought of giving probalities in selecting the random integer, but not sure
#  whether it is allowed to do or not , so left it having equal probs
def mutate(child):
    for i in range(len(child)):
        if random.random() < mutation_rate:
            child[i] = random.randint(0,4)
    return child

In [13]:
# Tried population 100,1000 -> 1000 seems taking a long time
n_population = 100
mutation_rate = 0.4 # After trying different mutation rates 0.4,0.5 seems reliable option
n_generations = 10000
n_parents = 20
steps = (2*maze_height + 1)*(2*maze_width + 1)
solution_path = []
population = generate_init_population(n_population, steps)

for generation in range(n_generations):
    fitness = get_fitness(population, maze)
    if generation%100 == 0:
        print('Generation 👶 :  ', generation)
        print('Best Fitness 🏋️ : ', min(fitness))
    if min(fitness) == 0:
        solution_path = population[fitness.index(0)]
        break
    population = sorted(population, key=lambda x: fitness[population.index(x)])
    population = population[:n_parents]
    children = get_next_generation(population)
    population = population + children
if solution_path == []:
    print('Solution not found! :(')
    print('Increasting population or generations might help 😐')
    print('Found Path till here: \n')
    maze.show_path(solution_path)
else:
  solution_path = population[fitness.index(min(fitness))]
  print("\n")
  print('Generation: ', generation)
  print('Best Fitness: ', min(fitness))
  print('Solution Path: \n', solution_path)
  print('Found the path!! 🥳 \n')
  maze.show_path(solution_path)



Generation 👶 :   0
Best Fitness 🏋️ :  10.5
Generation 👶 :   100
Best Fitness 🏋️ :  0.5


Generation:  112
Best Fitness:  0
Solution Path: 
 [3, 3, 4, 0, 0, 4, 0, 3, 3, 3, 3, 4, 0, 4, 0, 4, 4, 4, 4, 3, 4, 0, 4, 2, 4, 0, 1, 1, 4, 2, 3, 0, 3, 3, 4, 1, 1, 3, 0, 2, 2, 2, 1, 4, 4, 2, 4, 2, 2, 3, 2, 3, 0, 3, 1, 1, 2, 0, 0, 0, 4, 1, 1, 2, 3, 1, 0, 1, 4, 3, 3, 4, 2, 2, 1, 0, 2, 4, 0, 4, 3]
Found the path!! 🥳 

# # # # # # # # # 
# S #           # 
# 0 # # # # #   # 
# 0 0 0 #       # 
# # # 0 #   # # # 
#   # 0 #       # 
#   # 0 # # #   # 
#     0 0 0 0 E # 
# # # # # # # # # 

